In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.losses import BinaryCrossentropy

# a is the actual values while b is the predictions
def bce(a, b):
    return BinaryCrossentropy(from_logits = False)(a,b).numpy()

dtypes = ['train', 'validate', 'test']

2024-05-21 21:53:31.668876: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/envs/QPL/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [73]:
experiments = [0, 1, 2, 3, 4]
exp_path = './experiment_{}/'
pred_path = exp_path + 'predictions/'
sps_path = exp_path + 'processed_inputs_and_outputs/'

metric_functions = {'r': lambda x, y: np.corrcoef(x, y)[0,1],
                    'kl': lambda x, y: bce(x, y) - bce(x, x),
                    'l1': lambda x, y: np.mean(np.abs(x - y))
                    }
model_predictions = {'cnn': 'processed_cnn_predictions.npz', 'custom-nn': 'processed_custom_neural_net_predictions.npz',
                    'mirrored-cnn': 'processed_cnn_mirrored_predictions.npz', 'mirrored-custom-nn': 'processed_custom_neural_net_mirrored_predictions.npz'}

# Computing MAE performance

In [74]:
def aggregate_data(metric_function, model_types, circuits):
    device_results = {}
    for exp in experiments:
        device_results[exp] = {}
        if circuits != 'mirrored':
            infidelities = np.load(sps_path.format(exp) + 'processed_infidelities.npz')
        else:
            infidelities = np.load(sps_path.format(exp) + 'processed_mirrored_infidelities.npz')
        for model_type in model_types:
            predictions = np.load(pred_path.format(exp)+model_predictions[model_type])
            if model_type[0] != 'm':
                predictions = {dt: predictions[dt].reshape((-1,)) for dt in dtypes}
            else:
                predictions = predictions['predictions'].reshape((-1,))
            if model_type in ['custom-nn', 'cnn']:
                device_results[exp][model_type] = metric_function(infidelities[circuits].reshape((-1,))/100, predictions[circuits].reshape((-1,))/100)
            elif model_type in ['mirrored-custom-nn', 'mirrored-cnn']:
                device_results[exp][model_type] = metric_function(infidelities['infidelities'].reshape((-1,))/100, predictions.reshape((-1,))/100)
    return device_results

In [84]:
def compute_diffs(results: dict, model1: str, model2: str) -> dict:
    '''
    Loop over the entries in results and compute the difference between model1's entries and model2's entries.
    '''
    diff_dict = {exp: result[model1] - result[model2] for exp, result in results.items()}
    return diff_dict

def compute_relative_diffs(results: dict, model1: str, model2: str) -> dict:
    '''
    Loop over the entries in results and compute the difference between model1's entries and model2's entries.
    '''
    diff_dict = {exp: 100*(result[model1] - result[model2]) / result[model2] for exp, result in results.items()}
    return diff_dict

def compute_statistic(results: dict, models: list, statistic: np.mean, s_kwargs = {}) -> dict: 
    '''
    Loop over the devices in results and compute the (statistic) value of a metric for a model type.
    '''
    r_dict = {model: [] for model in models}
    mean_dict = {model: None for model in models}
    for _, result in results.items():
        for model in models:
            r_dict[model].append(result[model])
    for model, item in r_dict.items():
        mean_dict[model] = statistic(item, **s_kwargs)
    return mean_dict

Comparing on random circuits

In [88]:
model_types = ['custom-nn', 'cnn']
circuits = 'test'

m1, m2 = model_types
metrics = ['l1']
for metric in metrics:
    results = aggregate_data(metric_functions[metric], model_types, circuits)

m_dict = compute_statistic(results, [m1, m2], np.mean)
std_dict = compute_statistic(results, [m1, m2], np.std, {'ddof': 1})
results, m_dict, std_dict

({0: {'custom-nn': 0.17595568810144338, 'cnn': 0.40362890227131787},
  1: {'custom-nn': 0.19929601750709453, 'cnn': 0.4211730323686552},
  2: {'custom-nn': 0.19045894900771967, 'cnn': 0.4057395560832049},
  3: {'custom-nn': 0.19063093026996736, 'cnn': 0.36667339892587436},
  4: {'custom-nn': 0.19460001716886038, 'cnn': 0.4052834963861194}},
 {'custom-nn': 0.19018832041101708, 'cnn': 0.4004996772070344},
 {'custom-nn': 0.00873570518706385, 'cnn': 0.02019738858164003})

In [89]:
percent_diff_dict = compute_relative_diffs(results, m1, m2)
mean_percent_diff = np.mean([item for _, item in percent_diff_dict.items()])
std_percent_diff = np.std([item for _, item in percent_diff_dict.items()], ddof = 1)
mean_percent_diff, std_percent_diff

(-52.42820735008214, 3.0000116097852123)

Comparing on mirror circuits

In [94]:
model_types = ['mirrored-custom-nn', 'mirrored-cnn']
circuits = 'mirrored'

metrics = ['l1']
for metric in metrics:
    results = aggregate_data(metric_functions[metric], model_types, circuits)

m1, m2 = model_types

m_dict = compute_statistic(results, [m1, m2], np.mean)
std_dict = compute_statistic(results, [m1, m2], np.std, {'ddof': 1})
results, m_dict, std_dict

({0: {'mirrored-custom-nn': 0.7198123508255867,
   'mirrored-cnn': 1.1739220349082637},
  1: {'mirrored-custom-nn': 0.6521377312879126,
   'mirrored-cnn': 0.9220692764762998},
  2: {'mirrored-custom-nn': 0.7691960288610059,
   'mirrored-cnn': 1.2005252504890307},
  3: {'mirrored-custom-nn': 0.7194130564263412,
   'mirrored-cnn': 1.017424755623406},
  4: {'mirrored-custom-nn': 0.7612372097169209,
   'mirrored-cnn': 1.0458284406857643}},
 {'mirrored-custom-nn': 0.7243592754235534,
  'mirrored-cnn': 1.0719539516365528},
 {'mirrored-custom-nn': 0.04645268436689402,
  'mirrored-cnn': 0.11516234224226066})

In [95]:
percent_diff_dict = compute_relative_diffs(results, m1, m2)
mean_percent_diff = np.mean([item for _, item in percent_diff_dict.items()])
std_percent_diff = np.std([item for _, item in percent_diff_dict.items()], ddof = 1)
mean_percent_diff, std_percent_diff

(-32.077772103462245, 4.943646369244539)

# r value

Custom networks - random circuits

In [90]:
exps = [0, 1, 2, 3, 4]
ground = 'processed_infidelities.npz'
preds = 'processed_custom_neural_net_predictions.npz'
metric = 'r'

predictions = [np.load(pred_path.format(exp) + preds)['test'].reshape((-1,)) for exp in exps]
infidelities = [np.load(sps_path.format(exp) + ground)['test'].reshape((-1,)) for exp in exps]

metrics = [metric_functions[metric](p, i) for p,i in zip(predictions, infidelities)]
metrics, np.mean(metrics), np.std(metrics, ddof = 1)

([0.9684392681217231,
  0.9596715887004044,
  0.9702071654644208,
  0.9522838751667093,
  0.9599974257322668],
 0.9621198646371049,
 0.007290143789762289)

CNNs - random circuits

In [91]:
exps = [0, 1, 2, 3, 4]
ground = 'processed_infidelities.npz'
preds = 'processed_cnn_predictions.npz'
metric = 'r'

predictions = [np.load(pred_path.format(exp) + preds)['test'].reshape((-1,)) for exp in exps]
infidelities = [np.load(sps_path.format(exp) + ground)['test'].reshape((-1,)) for exp in exps]

metrics = [metric_functions[metric](p, i) for p,i in zip(predictions, infidelities)]
metrics, np.mean(metrics), np.std(metrics)

([0.7505361525027514,
  0.740882245477375,
  0.7316650834712786,
  0.7642203792560764,
  0.7625715799442024],
 0.7499750881303366,
 0.01248880119651289)

Custom networks - mirrored circuits

In [97]:
exps = [0, 1, 2, 3, 4]
ground = 'processed_mirrored_infidelities.npz'
preds = 'processed_custom_neural_net_mirrored_predictions.npz'
metric = 'r'

predictions = [np.load(pred_path.format(exp) + preds)['predictions'] for exp in exps]
infidelities = [np.load(sps_path.format(exp) + ground)['infidelities'] for exp in exps]

metrics = [metric_functions[metric](p, i) for p,i in zip(predictions, infidelities)]
metrics, np.mean(metrics), np.std(metrics, ddof = 1)

([0.9138791847764333,
  0.9211639497171052,
  0.9140251587778714,
  0.911793000949433,
  0.8979189324065388],
 0.9117560453254763,
 0.008509293767400028)

CNNs - mirrored circuits

In [78]:
exps = [0, 1, 2, 3, 4]
ground = 'processed_mirrored_infidelities.npz'
preds = 'processed_cnn_mirrored_predictions.npz'
metric = 'r'

predictions = [np.load(pred_path.format(exp) + preds)['predictions'] for exp in exps]
infidelities = [np.load(sps_path.format(exp) + ground)['infidelities'] for exp in exps]

metrics = [metric_functions[metric](p, i) for p,i in zip(predictions, infidelities)]
metrics, np.mean(metrics), np.std(metrics)

([0.8723350834502468,
  0.8606129905299384,
  0.8530316724664053,
  0.8569320381621299,
  0.8472077415570233],
 0.8580239052331488,
 0.008421604870836109)

# Circuit statistics

Random circuit depths

In [66]:
exps = [0, 1, 2, 3, 4]
exp_path = './experiment_{}/'
df_path = exp_path + '/simulation_results/high-fidelity-dataframe.csv'

In [67]:
def aggregate_data(metric: str) -> dict:
    results = {}
    for exp in exps:
        df = pd.read_csv(df_path.format(exp))
        min_metric, max_metric = min(df[metric]), max(df[metric])
        results[exp] = {'min': min_metric, 'max': max_metric}
    return results

In [68]:
depth_results = aggregate_data('F:Depth')
depth_results

{0: {'min': 1, 'max': 180},
 1: {'min': 1, 'max': 180},
 2: {'min': 1, 'max': 180},
 3: {'min': 1, 'max': 180},
 4: {'min': 1, 'max': 180}}

Mirror circuit depths

In [69]:
exps = [0, 1, 2, 3, 4]
exp_path = './experiment_{}/'
df_path = exp_path + '/simulation_results/mirrored_dataframe.csv'

In [70]:
m_depth_results = aggregate_data('F:Depth')
m_depth_results

{0: {'min': 8, 'max': 174},
 1: {'min': 7, 'max': 174},
 2: {'min': 9, 'max': 172},
 3: {'min': 7, 'max': 170},
 4: {'min': 7, 'max': 172}}

# Model size

In [128]:
from keras.models import load_model
from keras.optimizers import Adam
from keras.losses import MeanSquaredError

import json

from pygsti.extras import ml

exp_num = 1

model_path = f'./experiment_{exp_num}/models/'
exp_path = f'./experiment_{exp_num}/'

In [132]:
with open(exp_path + '/meta.json', 'r') as f:
    meta = json.load(f)

num_qubits = meta['num_qubits']
max_error_weight = meta['max_error_weight']
num_channels = meta['num_channels']
num_hops = meta['num_hops']
geometry = meta['geometry']

if meta['geometry'] == 'ring':
    adj_matrix = ml.newtools.ring_adj_matrix(num_qubits)
elif meta['geometry'] == 'algiers-t-bar':
    adj_matrix = ml.newtools.algiers_t_bar_adj_matrix()
elif meta['geometry'] == 't-bar':
    adj_matrix = ml.newtools.t_bar_adj_matrix()
elif meta['geometry'] == 'bowtie':
    adj_matrix = ml.newtools.bowtie_adj_matrix()
elif meta['geometry'] == 'melbourne':
    adj_matrix = ml.newtools.melbourne_adj_matrix()
else:
    print(f'You have not implemented the {geometry} geometry.')

laplace = ml.newtools.laplace_from_qubit_graph(adj_matrix)

error_gens = ml.newtools.up_to_weight_k_error_gens_from_qubit_graph(max_error_weight, num_qubits, laplace, num_hops)
error_gen_indices = [ml.newtools.error_gen_to_index(error[0], error[1]) for error in error_gens]


layer_snipper = ml.newneuralnets.layer_snipper_from_qubit_graph
layer_snipper_args = [num_qubits, num_channels, laplace, num_hops]

optimizer = Adam(learning_rate=1e-3)

circuits = np.load(exp_path+'/processed_inputs_and_outputs/processed_high_fidelity_circuits.npz')
infidelities = np.load(exp_path+'/processed_inputs_and_outputs/processed_infidelities.npz')

circuit_dense1 = ml.newneuralnets.CircuitErrorVec(num_qubits, num_channels, error_gens, layer_snipper,
                                                layer_snipper_args)

circuit_dense1.predict(circuits['train'][0:1])

circuit_dense1.summary()

# circuit_dense.load_weights(model_path.format(device) + 'custom_neural_network')

1/1 [==============================] - 18s 18s/step
Model: "circuit_error_vec_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [None]                    0         
                                                                 
 localized_dense_to_err_vec_  multiple                 299772    
 2 (LocalizedDenseToErrVec)                                      
                                                                 
Total params: 299,772
Trainable params: 299,772
Non-trainable params: 0
_________________________________________________________________
